In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from rapidfuzz import process
from datetime import datetime, timedelta
import re
import os

### Rotowire.com scores

In [ ]:
def teams_matchups(game_date):
    # Load the options
    options = Options()
    options.add_argument("--headless")  # Optional: Run in headless mode
    options.binary_location = "C:\\Program Files\\BraveSoftware\\Brave-Browser\\Application\\brave.exe"

    # Set up the WebDriver
    driver = webdriver.Chrome(options= options)    
    driver.get(f"https://www.rotowire.com/baseball/scoreboard.php?date={game_date}")

    datatable_id = 'grid-noGutter mb-15'

    # Explicitly wait for the table element to load
    datatable_xpath = f"//div[@class='{datatable_id}']"  # Update XPATH as needed
    try:
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, datatable_xpath))
        )
        print(f"{datatable_id} table loaded successfully.")
    except Exception as e:
        print(f"Error: Table {datatable_id} did not load. Details: {e}")
        driver.quit()

    # Wait for the load of the page
    time.sleep(10)

    # Locate the table
    table_element = driver.find_element(By.XPATH, datatable_xpath)
    text_content = table_element.text

    # Process the table content
    rows = text_content.split("\n")
    table_data = [row.split("\t") for row in rows]

    # Convert to dataframe
    df = pd.DataFrame(table_data)

    # Find indices for where "Final" and "View Box Score" appear
    start_indices = df[df[0].str.contains("Final", case=False)].index
    end_indices = df[df[0].str.contains("View Box Score", case=False)].index

    # Extract and split data
    game_dataframes = []
    for start in start_indices:
        # Find the corresponding end index that's greater than the start index
        end = end_indices[end_indices > start].min()
        if pd.notna(end):  # Ensure there's a valid end index
            game_data = df.iloc[start:end+1]  # Capture all rows in between
            # Split every 13 rows and create DataFrame
            reshaped_data = [game_data.iloc[i:i+13] for i in range(0, len(game_data), 13)]
            game_dataframes.extend(reshaped_data)

    # Final dataframe with all games
    final_df = pd.concat(game_dataframes, ignore_index= True)

    # Convert the DataFrame to a numpy array for reshaping
    data = final_df.values  

    # Reshape: each group of 13 rows becomes one row with 13 columns
    reshaped_data = [data[i:i+13].flatten() for i in range(0, len(data), 13)]

    # Convert reshaped data back to a DataFrame
    reshaped_df = pd.DataFrame(reshaped_data)

    # Drop the first 4 columns and last column
    reshaped_df = reshaped_df.drop(reshaped_df.columns[[0, 1, 2, 3, -1]], axis= 1)

    # Add the headers
    reshaped_df.columns = ['Away', 'Home', 'R_Away', 'H_Away', 'E_Away', 'R_Home', 'H_Home', 'E_Home']

    # Add the date
    reshaped_df['date'] = game_date

    # Calculate the winner, loser and the difference in runs, hits and errors
    reshaped_df['winner']    = reshaped_df.apply(lambda x: x['Away'] if int(x['R_Away']) > int(x['R_Home']) else x['Home'], axis= 1)
    reshaped_df['loser']     = reshaped_df.apply(lambda x: x['Away'] if int(x['R_Away']) < int(x['R_Home']) else x['Home'], axis= 1)
    reshaped_df['diff_runs_away_vs_home_team']   = reshaped_df.apply(lambda x: int(x['R_Away']) - int(x['R_Home']), axis= 1)
    reshaped_df['diff_hits_away_vs_home_team']   = reshaped_df.apply(lambda x: int(x['H_Away']) - int(x['H_Home']), axis= 1)
    reshaped_df['diff_errors_away_vs_home_team'] = reshaped_df.apply(lambda x: int(x['E_Away']) - int(x['E_Home']), axis= 1)
    
    return reshaped_df

# Define dates
dates = [
    (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d'),
    (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d'),
    (datetime.now() - timedelta(days=3)).strftime('%Y-%m-%d'),
    (datetime.now() - timedelta(days=4)).strftime('%Y-%m-%d'),
    (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d'),
    (datetime.now() - timedelta(days=6)).strftime('%Y-%m-%d'),
    (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
    ]

# Initialize a dictionary to store dataframes
dataframes = {}

# Loop through each date and store results in a unique dataframe
for game_date in dates:
    print(f"Processing data for date: {game_date}")
    
    # Run the teams_matchups function and save the resulting dataframe
    result_df = teams_matchups(game_date)
    
    # Store the dataframe in the dictionary with the date as the key
    dataframes[game_date] = result_df

# Create the final dataframe
games = pd.concat(dataframes.values(), ignore_index=True)

# Convert to datetime format
games["date"] = pd.to_datetime(games["date"])

# Create a game_id column using the index
games["game_id"] = games.index + 1
games["game_id"] = games["game_id"].astype(str)

# Create a key column for the game
games["key"] = games["date"].dt.strftime("%Y%m%d") + "_" + games["game_id"]

# Know if the winner was the away or home team
games["visitor_won"]           = games.apply(lambda x: 1 if int(x["R_Away"]) > int(x["R_Home"]) else 0, axis=1)
games["home_won"]              = games.apply(lambda x: 1 if int(x["R_Away"]) < int(x["R_Home"]) else 0, axis=1) 
games["visit_or_home_victory"] = games.apply(lambda x: 'H' if int(x["R_Away"]) < int(x["R_Home"]) else 'V', axis=1) 

# Create a column that indicates if the game was a shutout
games["shutout"] = games.apply(lambda x: 1 if int(x["R_Away"]) == 0 or int(x["R_Home"]) == 0 else 0, axis=1)

# Create a column that indicates if the game was a one-run game
games["one_run_game"] = games.apply(lambda x: 1 if abs(int(x["R_Away"]) - int(x["R_Home"])) == 1 else 0, axis=1)

# Create a column that indicates if the game was a high-scoring game
games["high_scoring_game"] = games.apply(lambda x: 1 if int(x["R_Away"]) + int(x["R_Home"]) >= 10 else 0, axis=1)

# Create a column that indicates if the game was a low-scoring game
games["low_scoring_game"] = games.apply(lambda x: 1 if int(x["R_Away"]) + int(x["R_Home"]) <= 3 else 0, axis=1)

# Create a column that indicates if the game was a blowout
games["blowout"] = games.apply(lambda x: 1 if abs(int(x["R_Away"]) - int(x["R_Home"])) >= 5 else 0, axis=1)

# Create a column that indicates how many runs were scored in the game
games["total_runs"] = games.apply(lambda x: int(x["R_Away"]) + int(x["R_Home"]), axis=1)

# Create a column that indicates how many hits were scored in the game
games["total_hits"] = games.apply(lambda x: int(x["H_Away"]) + int(x["H_Home"]), axis=1)

# Create a column that indicates how many errors were scored in the game
games["total_errors"] = games.apply(lambda x: int(x["E_Away"]) + int(x["E_Home"]), axis=1)

# Create a column that join the home and away teams
games["teams"] = games.apply(lambda x: x["Away"] + " vs " + x["Home"], axis=1)

# Count occurrences of each team matchup in the 'teams' column
team_counts = games['teams'].value_counts()

# Map the counts back to the original dataFrame
games['team_matchup_count'] = games['teams'].map(team_counts)

# Create a group id for each team matchup
games['series_id'] = games.groupby('teams').ngroup() + 1

# Export the dataframe to a CSV file
games.to_csv('D:\\mlb_analyzer\\output\\teams\\teams_matchup.csv', index=False)


## Basic pitcher information

In [ ]:
def games_today():
    """Get today's matchups from the RotoWire scoreboard.
    This function uses Selenium to scrape the RotoWire website for today's MLB matchups.
    It extracts the game time, away team, home team, away pitcher name, away pitcher record,
    home pitcher name, and home pitcher record from the scoreboard.
    The data is then reshaped into a DataFrame format for further analysis.

    Returns:
        _type_: _description_
    """
    # Load the options
    options = Options()
    options.add_argument("--headless")  # Optional: Run in headless mode
    options.binary_location = "C:\\Program Files\\BraveSoftware\\Brave-Browser\\Application\\brave.exe"

    today = datetime.now().strftime('%Y-%m-%d')

    # Set up the WebDriver
    driver = webdriver.Chrome(options= options)    
    driver.get(f"https://www.rotowire.com/baseball/scoreboard.php?date={today}")

    datatable_id = 'grid-noGutter mb-15'

    # Explicitly wait for the table element to load
    datatable_xpath = f"//div[@class='{datatable_id}']"  # Update XPATH as needed
    try:
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, datatable_xpath))
        )
        print(f"{datatable_id} table loaded successfully.")
    except Exception as e:
        print(f"Error: Table {datatable_id} did not load. Details: {e}")
        driver.quit()

    # Wait for the load of the page
    time.sleep(10)

    # Locate the table
    table_element = driver.find_element(By.XPATH, datatable_xpath)
    text_content = table_element.text

    # Process the table content
    rows = text_content.split("\n")
    table_data = [row.split("\t") for row in rows]

    # Convert to dataframe
    df = pd.DataFrame(table_data)

    # Find indices for where "Final" and "View Box Score" appear
    start_indices = df[df[0].str.contains(" ET", case= False)].index
    end_indices = df[df[0].str.contains("View Box Score", case=False)].index

    # Extract and split data
    game_dataframes = []
    for start in start_indices:
        # Find the corresponding end index that's greater than the start index
        end = end_indices[end_indices > start].min()
        if pd.notna(end):  # Ensure there's a valid end index
            game_data = df.iloc[start:end+1]  # Capture all rows in between
            # Split every 8 rows and create DataFrame
            reshaped_data = [game_data.iloc[i:i+8] for i in range(0, len(game_data), 8)]
            game_dataframes.extend(reshaped_data)

    # Final dataframe with all games
    final_df = pd.concat(game_dataframes, ignore_index= True)

    # Convert the DataFrame to a numpy array for reshaping
    data = final_df.values  

    # Reshape: each group of 8 rows becomes one row with 8 columns
    reshaped_data = [data[i:i+8].flatten() for i in range(0, len(data), 8)]

    # Convert reshaped data back to a DataFrame
    reshaped_df = pd.DataFrame(reshaped_data)

    # Drop the last column
    reshaped_df = reshaped_df.drop(reshaped_df.columns[[-1]], axis= 1)

    # Add the headers
    reshaped_df.columns = ['game_time', 'away_team', 'home_team', 'away_pitcher_name', 'away_pitcher_record', 'home_pitcher_name', 'home_pitcher_record']

    # Add the date
    reshaped_df['date'] = today
    
    return reshaped_df

# Get today's matchups
games_today_df = games_today()

# Convert the date column to datetime format
games_today_df["date"] = pd.to_datetime(games_today_df["date"])

# Create a game_id column using the index
games_today_df["game_id"] = games_today_df.index + 1
games_today_df["game_id"] = games_today_df["game_id"].astype(str)

# Export the dataframe to a CSV file
games_today_df.to_csv('D:\\mlb_analyzer\\output\\teams\\games_today.csv', index=False)


## Advanced Matchups

In [ ]:
def advanced_matchups():
    # Load the options
    options = Options()
    options.add_argument("--headless")  # Optional: Run in headless mode
    options.binary_location = "C:\\Program Files\\BraveSoftware\\Brave-Browser\\Application\\brave.exe"

    today = datetime.now().strftime('%Y-%m-%d')

    # Set up the WebDriver
    driver = webdriver.Chrome(options= options)    
    driver.get(f"https://baseballsavant.mlb.com/probable-pitchers")

    datatable_id = 'template__content template--two-column__content--one'

    # Explicitly wait for the table element to load
    datatable_xpath = f"//div[@class='{datatable_id}']"  # Update XPATH as needed
    try:
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, datatable_xpath))
        )
        print(f"{datatable_id} table loaded successfully.")
    except Exception as e:
        print(f"Error: Table {datatable_id} did not load. Details: {e}")
        driver.quit()

    # Wait for the load of the page
    time.sleep(10)

    # Locate the table
    table_element = driver.find_element(By.XPATH, datatable_xpath)
    text_content = table_element.text

    # Process the table content
    rows = text_content.split("\n")
    table_data = [row.split("\t") for row in rows]

    # Convert to dataframe
    df = pd.DataFrame(table_data)

    # Identify rows that contain " @ "
    split_indices = df[df[0].str.contains(" @ ")].index.tolist()

    # Add start and end indices
    split_indices.append(len(df))
    split_data = [df.iloc[split_indices[i]:split_indices[i+1]].values.flatten().tolist()
                    for i in range(len(split_indices)-1)]

    # Create new DataFrame
    new_df = pd.DataFrame(split_data)

    # Splitting on the " | " symbol. Splitting only col 2 while keeping other columns
    df_expanded = new_df.copy()  # Preserve other columns
    df_expanded[['Column1', 'Column2']] = df_expanded[2].str.split("ET", expand=True)

    # Drop original column
    df_expanded = df_expanded.drop(columns=[2, 3])

    #! Removing rows where col 4 contains 'to be announced'
    df_filtered = df_expanded[df_expanded[4] != "To be announced."]

    # Filtering rows where col6 contains "Never Faced Any Players on this Team."
    df_never_faced_the_team = df_filtered[df_filtered[6] == "Never Faced Any Players on this Team."].copy()

    # Removing rows from the filtered DataFrame
    df_filtered = df_filtered[df_filtered[6] != "Never Faced Any Players on this Team."]

    # Filtering rows where col9 does not contains "Exit Velo Launch Angle xBA xSLG xwOBA"
    df_not_complete = df_filtered[df_filtered[9] != "Exit Velo Launch Angle xBA xSLG xwOBA"].copy()

    # Removing rows from the filtered DataFrame
    df_filtered = df_filtered[df_filtered[9] == "Exit Velo Launch Angle xBA xSLG xwOBA"]

    # Convert empty strings to NaN for better handling
    df_filtered[17] = df_filtered[17].replace("", pd.NA)

    # Count occurrences of each unique non-empty value
    value_counts = df_filtered[17].dropna().value_counts()

    if not value_counts.empty:
        # Identify the most frequent value
        most_frequent_value = value_counts.idxmax()

        # Fill NaN values with the most frequent value
        df_filtered[17] = df_filtered[17].fillna(most_frequent_value)

    # Splitting on the " ET ". Splitting only col 2 while keeping other columns
    df_expanded = new_df.copy()  # Preserve other columns
    df_expanded[['Column1', 'Column2']] = df_expanded[2].str.split("ET", expand=True)

    # Drop original column
    df_expanded = df_expanded.drop(columns=[2, 3])

    # Splitting columns
    df_split = df_filtered.copy()  # Preserve other columns
    df_split[['PA_away_pitcher', 'K%_away_pitcher', 'BB%_away_pitcher', 'AVG_away_pitcher', 'wOBA_away_pitcher']] = df_split[8].str.split(" ", expand=True)
    df_split[['Exit_Velo_away_pitcher', 'unit_away', 'Lunch_Angle_away_pitcher', 'xBA_away_pitcher', 'xSLG_away_pitcher', 'xwOBA_away_pitcher']] = df_split[10].str.split(" ", expand=True)

    df_split[['PA_home_pitcher', 'K%_home_pitcher', 'BB%_home_pitcher', 'AVG_home_pitcher', 'wOBA_home_pitcher']] = df_split[16].str.split(" ", expand=True)
    df_split[['Exit_Velo_home_pitcher', 'unit_home', 'Lunch_Angle_home_pitcher', 'xBA_home_pitcher', 'xSLG_home_pitcher', 'xwOBA_home_pitcher']] = df_split[18].str.split(" ", expand=True)

    # Drop original columns if needed
    df_split = df_split.drop(columns=[6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19])

    # Function to remove city names
    def remove_city_names(text):
        return re.sub(r'\b(?:San Diego|Pittsburgh|Arizona|Philadelphia|Kansas City|Baltimore|Tampa Bay|New York|Cleveland|Toronto|Minnesota|Boston|Los Angeles|Atlanta|Houston|Chicago|Seattle|Texas|Milwaukee|St. Louis|Detroit|Colorado|San Francisco)\b ', '', text)

    # Apply the function to the first column of the DataFrame
    df_split[0] = df_split[0].apply(remove_city_names)

    # Replace @ in col 0 with "vs"
    df_split[0] = df_split[0].str.replace(" @ ", " vs ", regex= True)

    # Splitting on the " vs ". Splitting only col 0 while keeping other columns.
    df_split = df_split.copy()  # Preserve other columns
    df_split[['away_team', 'home_team']] = df_split[0].str.split(" vs ", expand=True)

    # Update the column with the date
    df_split[1] = datetime.now().strftime('%Y-%m-%d')

    # # Add the headers
    # new_df.columns = ['teams', 'date', 'time_and_park', 'away_pitcher_name', 'away_pitcher_throws', 'away_pitcher_info', 
    #                     'away_pitcher_fields_1', 'away_pitcher_data_1', 'away_pitcher_fields_2', 'away_pitcher_data_2',]
    
    return df_split, df_never_faced_the_team, df_not_complete


# Call the function to get today's matchups with advanced stats
advanced_matchups_df, never_faced_the_team_df, df_not_complete = advanced_matchups()

# Export the dataframe to a CSV file
advanced_matchups_df.to_csv('D:\\mlb_analyzer\\output\\teams\\advanced_matchups\\advanced_matchups.csv', index=False)
never_faced_the_team_df.to_csv('D:\\mlb_analyzer\\output\\teams\\advanced_matchups\\never_faced_the_team.csv', index=False)
df_not_complete.to_csv('D:\\mlb_analyzer\\output\\teams\\advanced_matchups\\not_complete.csv', index=False)


# Import advanced data and gamelogs for each team

#### This works for hitting and pitching

In [ ]:
# def team_advanced_stats(analysis_type):
#     """Get the advanced stats for the team.
#     The function scrapes the advanced stats from the MLB website using Selenium and returns three dataframes:
#     statcast, plate_discipline and batted_ball_profile.

#     Returns:
#         _type_: _description_
#     """
#     # Load the options
#     options = Options()
#     options.add_argument("--headless")  # Optional: Run in headless mode
#     options.binary_location = "C:\\Program Files\\BraveSoftware\\Brave-Browser\\Application\\brave.exe"

#     year = datetime.now().year

#     # Set up the WebDriver
#     driver = webdriver.Chrome(options= options)    
#     driver.get(f"https://baseballsavant.mlb.com/team/114?view=statcast&nav={analysis_type}&season={year}")

#     datatable_id = 'div_statcast'
#     datatable_xpath = f"//div[@id='{datatable_id}']"  # Update XPATH as needed

#     # Initialize an empty dataframe
#     team_advanced_stats_df = pd.DataFrame()

#     try:
#         # Explicitly wait for the table element to load
#         WebDriverWait(driver, 20).until(
#             EC.presence_of_element_located((By.XPATH, datatable_xpath))
#         )
#         print(f"{datatable_id} table loaded successfully.")

#         # Locate the table
#         table_element = driver.find_element(By.XPATH, datatable_xpath)
#         text_content = table_element.text

#         # Process the table content
#         rows = text_content.split("\n")
#         table_data = [row.split("\t") for row in rows]

#         # Convert to dataframe
#         team_advanced_stats_df = pd.DataFrame(table_data)

#     except Exception as e:
#         print(f"Error: Table {datatable_id} did not load. Returning an empty dataframe. Details: {e}")

#     finally:
#         driver.quit()
    
#     # Check if the dataframe is empty
#     # If the dataframe is empty, return empty dataframes
#     if team_advanced_stats_df.empty:
#         print("No data found. Skipping table creation.")        
#         return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()  # Return empty dataframes
#     else:
        
#         #! STATCAST TABLE
#         #! Cleaning the data for the statcast table


#         def combine_rows(df, row_number_1, row_number_2):
#             """Combine two rows in a DataFrame into one row.
#             The first row will contain the combined data, and the second row will be dropped.

#             Args:
#                 df (_type_): _description_
#                 row_number_1 (_type_): _description_
#                 row_number_2 (_type_): _description_

#             Returns:
#                 _type_: _description_
#             """
#             # Combine the two rows
#             df.loc[row_number_1, 0] = df.loc[row_number_1, 0] + ' ' + df.loc[row_number_2, 0]

#             # Drop the second row
#             df = df.drop(row_number_2).reset_index(drop=True)
            
#             return df


#         # Join rows
#         team_advanced_stats_df = combine_rows(team_advanced_stats_df, 19, 20)
#         team_advanced_stats_df = combine_rows(team_advanced_stats_df, 23, 24)
#         team_advanced_stats_df = combine_rows(team_advanced_stats_df, 24, 25)

#         #! Create the headers for the first table (statcast)
#         # Find the first occurrence of 'Player' and 'XWOBACON'
#         # Find indices
#         start_idx = team_advanced_stats_df[team_advanced_stats_df[0] == 'Player'].index[0]
#         end_idx = team_advanced_stats_df[team_advanced_stats_df[0] == 'XWOBACON'].index[0]

#         # Slice and transpose
#         headers_statcast = team_advanced_stats_df.iloc[start_idx:end_idx + 1].T

#         # Reset column names
#         headers_statcast.columns = headers_statcast.iloc[0]
#         headers_statcast         = headers_statcast[1:].reset_index(drop=True)

#         # Remove those rows from original dataframe
#         team_advanced_stats_df = team_advanced_stats_df.drop(team_advanced_stats_df.index[start_idx:end_idx + 1]).reset_index(drop=True)

#         # Remove the first 3 rows
#         team_advanced_stats_df = team_advanced_stats_df.iloc[3:].reset_index(drop=True)  # Using iloc

#         # Use regex to split the column while preserving negative numbers
#         team_advanced_stats_df[['Name', 'Numbers']] = team_advanced_stats_df[0].str.extract(r'^(.*?)([-\d\s.,]*)$')

#         # Split the 'Numbers' column into separate columns (25 columns)
#         team_advanced_stats_df = team_advanced_stats_df.join(team_advanced_stats_df['Numbers'].str.split(expand=True).rename(lambda x: f'col_{x+1}', axis=1))

#         # Drop the original 'Numbers' column
#         team_advanced_stats_df = team_advanced_stats_df.drop(columns=['Numbers'])

#         # Find the first empty row
#         first_empty_idx = team_advanced_stats_df[team_advanced_stats_df[0] == ''].index.min()

#         # Extract rows from the start until the first empty row
#         statcast = team_advanced_stats_df.iloc[:first_empty_idx]

#         # Drop the first column
#         statcast = statcast.drop(columns=[0])

#         # Swap last name and first name
#         statcast['Name'] = statcast['Name'].str.split(', ').str[::-1].str.join(' ')

#         # Add the headers to the dataframe
#         statcast.columns = headers_statcast.columns

#         #! PLATE DISCIPLINE TABLE
#         #! Cleaning the data for the plate discipline table
#         # Find the first empty row
#         first_empty_idx = team_advanced_stats_df[team_advanced_stats_df[0] == ''].index.min()

#         # Remove rows from the first row until the first empty row
#         team_advanced_stats_df = team_advanced_stats_df.iloc[first_empty_idx + 1:].reset_index(drop=True)

#         # Remove the first row
#         team_advanced_stats_df = team_advanced_stats_df.iloc[1:].reset_index(drop= True)  # Using iloc

#         # Join rows
#         team_advanced_stats_df = combine_rows(team_advanced_stats_df, 4, 5)
#         team_advanced_stats_df = combine_rows(team_advanced_stats_df, 5, 6)
#         team_advanced_stats_df = combine_rows(team_advanced_stats_df, 7, 8)
#         team_advanced_stats_df = combine_rows(team_advanced_stats_df, 9, 10)
#         team_advanced_stats_df = combine_rows(team_advanced_stats_df, 13, 14)

#         #! Create the headers for the second table (plate discipline)
#         # Find the first occurrence of 'Player' and 'Meatball Swing %'
#         # Find indices
#         start_idx = team_advanced_stats_df[team_advanced_stats_df[0] == 'Player'].index[0]
#         end_idx = team_advanced_stats_df[team_advanced_stats_df[0] == 'Meatball Swing %'].index[0]

#         # Slice and transpose
#         headers_plate_discipline = team_advanced_stats_df.iloc[start_idx:end_idx + 1].T

#         # Reset column names
#         headers_plate_discipline.columns = headers_plate_discipline.iloc[0]
#         headers_plate_discipline         = headers_plate_discipline[1:].reset_index(drop=True)

#         # Remove those rows from original dataframe
#         team_advanced_stats_df = team_advanced_stats_df.drop(team_advanced_stats_df.index[start_idx:end_idx + 1]).reset_index(drop=True)

#         # Find the first empty row
#         first_empty_idx = team_advanced_stats_df[team_advanced_stats_df[0] == ''].index.min()

#         # Extract rows from the start until the first empty row
#         plate_discipline = team_advanced_stats_df.iloc[:first_empty_idx]

#         # Drop the first column
#         plate_discipline = plate_discipline.drop(columns=[0])

#         # Swap last name and first name
#         plate_discipline['Name'] = plate_discipline['Name'].str.split(', ').str[::-1].str.join(' ')

#         # Drop columns where all values are NaN (empty)
#         plate_discipline = plate_discipline.dropna(axis= 1, how= 'all')

#         # Add the headers to the dataframe
#         plate_discipline.columns = headers_plate_discipline.columns

#         #! BATTED BALL PROFILE TABLE
#         #! Cleaning the data for the batted ball profile table
#         # Find the first empty row
#         first_empty_idx = team_advanced_stats_df[team_advanced_stats_df[0] == ''].index.min()

#         # Remove rows from the first row until the first empty row
#         team_advanced_stats_df = team_advanced_stats_df.iloc[first_empty_idx + 1:].reset_index(drop=True)

#         # Remove the first row
#         team_advanced_stats_df = team_advanced_stats_df.iloc[1:].reset_index(drop= True)  # Using iloc

#         # Join rows
#         team_advanced_stats_df = combine_rows(team_advanced_stats_df, 15, 16)

#         #! Create the headers for the third table (batted ball profile)
#         # Find the first occurrence of 'Player' and 'Barrel %'
#         # Find indices
#         start_idx = team_advanced_stats_df[team_advanced_stats_df[0] == 'Player'].index[0]
#         end_idx = team_advanced_stats_df[team_advanced_stats_df[0] == 'Barrel %'].index[0]

#         # Slice and transpose
#         headers_batted_ball_profile = team_advanced_stats_df.iloc[start_idx:end_idx + 1].T

#         # Reset column names
#         headers_batted_ball_profile.columns = headers_batted_ball_profile.iloc[0]
#         headers_batted_ball_profile         = headers_batted_ball_profile[1:].reset_index(drop=True)

#         # Remove those rows from original dataframe
#         team_advanced_stats_df = team_advanced_stats_df.drop(team_advanced_stats_df.index[start_idx:end_idx + 1]).reset_index(drop=True)

#         # Find the first empty row
#         first_empty_idx = team_advanced_stats_df[team_advanced_stats_df[0] == ''].index.min()

#         if pd.isna(first_empty_idx):
#             batted_ball_profile = team_advanced_stats_df.copy()  # If no empty row, use the entire DataFrame
#         else:
#             # Extract rows from the start until the first empty row
#             batted_ball_profile = team_advanced_stats_df.iloc[:first_empty_idx]

#         # Drop the first column
#         batted_ball_profile = batted_ball_profile.drop(columns=[0])

#         # Swap last name and first name
#         batted_ball_profile['Name'] = batted_ball_profile['Name'].str.split(', ').str[::-1].str.join(' ')

#         # Drop columns where all values are NaN (empty)
#         batted_ball_profile = batted_ball_profile.dropna(axis= 1, how= 'all')

#         # Add the headers to the dataframe
#         batted_ball_profile.columns = headers_batted_ball_profile.columns
        
#         return statcast, plate_discipline, batted_ball_profile


# # Call the function to get today's matchups with advanced stats
# #hitting_statcast_df,  hitting_plate_discipline_df,  hitting_batted_ball_profile_df  = team_advanced_stats(analysis_type= 'hitting')
# pitching_statcast_df, pitching_plate_discipline_df, pitching_batted_ball_profile_df = team_advanced_stats(analysis_type= 'pitching')


In [ ]:

# #! STATCAST TABLE
# #! Cleaning the data for the statcast table

# def combine_rows(df, row_number_1, row_number_2):
#     """Combine two rows in a DataFrame into one row.
#     The first row will contain the combined data, and the second row will be dropped.

#     Args:
#         df (_type_): _description_
#         row_number_1 (_type_): _description_
#         row_number_2 (_type_): _description_

#     Returns:
#         _type_: _description_
#     """
#     # Combine the two rows
#     df.loc[row_number_1, 0] = df.loc[row_number_1, 0] + ' ' + df.loc[row_number_2, 0]

#     # Drop the second row
#     df = df.drop(row_number_2).reset_index(drop=True)
    
#     return df


# # Join rows
# test_df = combine_rows(test_df, 19, 20)
# test_df = combine_rows(test_df, 23, 24)
# test_df = combine_rows(test_df, 24, 25)

# #! Create the headers for the first table (statcast)
# # Find the first occurrence of 'Player' and 'XWOBACON'
# # Find indices
# start_idx = test_df[test_df[0] == 'Player'].index[0]
# end_idx = test_df[test_df[0] == 'XWOBACON'].index[0]

# # Slice and transpose
# headers_statcast = test_df.iloc[start_idx:end_idx + 1].T

# # Reset column names
# headers_statcast.columns = headers_statcast.iloc[0]
# headers_statcast         = headers_statcast[1:].reset_index(drop=True)

# # Remove those rows from original dataframe
# test_df = test_df.drop(test_df.index[start_idx:end_idx + 1]).reset_index(drop=True)

# # Remove the first 3 rows
# test_df = test_df.iloc[3:].reset_index(drop=True)  # Using iloc

# # Use regex to split the column while preserving negative numbers
# test_df[['Name', 'Numbers']] = test_df[0].str.extract(r'^(.*?)([-\d\s.,]*)$')

# # Split the 'Numbers' column into separate columns (25 columns)
# test_df = test_df.join(test_df['Numbers'].str.split(expand=True).rename(lambda x: f'col_{x+1}', axis=1))

# # Drop the original 'Numbers' column
# test_df = test_df.drop(columns=['Numbers'])

# # Find the first empty row
# first_empty_idx = test_df[test_df[0] == ''].index.min()

# # Extract rows from the start until the first empty row
# statcast = test_df.iloc[:first_empty_idx]

# # Drop the first column
# statcast = statcast.drop(columns=[0])

# # Swap last name and first name
# statcast['Name'] = statcast['Name'].str.split(', ').str[::-1].str.join(' ')

# # Add the headers to the dataframe
# statcast.columns = headers_statcast.columns


In [ ]:

# #! PLATE DISCIPLINE TABLE
# #! Cleaning the data for the plate discipline table
# # Find the first empty row
# first_empty_idx = test_df[test_df[0] == ''].index.min()

# # Remove rows from the first row until the first empty row
# test_df = test_df.iloc[first_empty_idx + 1:].reset_index(drop=True)

# # Remove the first row
# test_df = test_df.iloc[1:].reset_index(drop= True)  # Using iloc

# # Join rows
# test_df = combine_rows(test_df, 4, 5)
# test_df = combine_rows(test_df, 5, 6)
# test_df = combine_rows(test_df, 7, 8)
# test_df = combine_rows(test_df, 9, 10)
# test_df = combine_rows(test_df, 13, 14)

# #! Create the headers for the second table (plate discipline)
# # Find the first occurrence of 'Player' and 'Meatball Swing %'
# # Find indices
# start_idx = test_df[test_df[0] == 'Player'].index[0]
# end_idx = test_df[test_df[0] == 'Meatball Swing %'].index[0]

# # Slice and transpose
# headers_plate_discipline = test_df.iloc[start_idx:end_idx + 1].T

# # Reset column names
# headers_plate_discipline.columns = headers_plate_discipline.iloc[0]
# headers_plate_discipline         = headers_plate_discipline[1:].reset_index(drop=True)

# # Remove those rows from original dataframe
# test_df = test_df.drop(test_df.index[start_idx:end_idx + 1]).reset_index(drop=True)

# # Find the first empty row
# first_empty_idx = test_df[test_df[0] == ''].index.min()

# # Extract rows from the start until the first empty row
# plate_discipline = test_df.iloc[:first_empty_idx]

# # Drop the first column
# plate_discipline = plate_discipline.drop(columns=[0])

# # Swap last name and first name
# plate_discipline['Name'] = plate_discipline['Name'].str.split(', ').str[::-1].str.join(' ')

# # Drop columns where all values are NaN (empty)
# plate_discipline = plate_discipline.dropna(axis= 1, how= 'all')

# # Add the headers to the dataframe
# plate_discipline.columns = headers_plate_discipline.columns


In [ ]:

# #! BATTED BALL PROFILE TABLE
# #! Cleaning the data for the batted ball profile table
# # Find the first empty row
# first_empty_idx = test_df[test_df[0] == ''].index.min()

# # Remove rows from the first row until the first empty row
# test_df = test_df.iloc[first_empty_idx + 1:].reset_index(drop=True)

# # Remove the first row
# test_df = test_df.iloc[1:].reset_index(drop= True)  # Using iloc

# # Join rows
# test_df = combine_rows(test_df, 15, 16)

# #! Create the headers for the third table (batted ball profile)
# # Find the first occurrence of 'Player' and 'Barrel %'
# # Find indices
# start_idx = test_df[test_df[0] == 'Player'].index[0]
# end_idx = test_df[test_df[0] == 'Barrel %'].index[0]

# # Slice and transpose
# headers_batted_ball_profile = test_df.iloc[start_idx:end_idx + 1].T

# # Reset column names
# headers_batted_ball_profile.columns = headers_batted_ball_profile.iloc[0]
# headers_batted_ball_profile         = headers_batted_ball_profile[1:].reset_index(drop=True)

# # Remove those rows from original dataframe
# test_df = test_df.drop(test_df.index[start_idx:end_idx + 1]).reset_index(drop=True)

# # Find the first empty row
# first_empty_idx = test_df[test_df[0] == ''].index.min()

# if pd.isna(first_empty_idx):
#     batted_ball_profile = test_df.copy()  # If no empty row, use the entire DataFrame
# else:
#     # Extract rows from the start until the first empty row
#     batted_ball_profile = test_df.iloc[:first_empty_idx]

# # Drop the first column
# batted_ball_profile = batted_ball_profile.drop(columns=[0])

# # Swap last name and first name
# batted_ball_profile['Name'] = batted_ball_profile['Name'].str.split(', ').str[::-1].str.join(' ')

# # Drop columns where all values are NaN (empty)
# batted_ball_profile = batted_ball_profile.dropna(axis= 1, how= 'all')

# # Add the headers to the dataframe
# batted_ball_profile.columns = headers_batted_ball_profile.columns

In [ ]:
def import_advanced_stats(analysis_type, team_ids, category):
    """Get the advanced stats for multiple teams.
    The function scrapes the advanced stats from the MLB website using Selenium and returns a dictionary of dataframes.

    Returns:
        dict: A dictionary mapping team IDs to their respective advanced stats dataframe.
    """
    options = Options()
    options.add_argument("--headless")  # Optional: Run in headless mode
    options.binary_location = "C:\\Program Files\\BraveSoftware\\Brave-Browser\\Application\\brave.exe"

    year = datetime.now().year
    driver = webdriver.Chrome(options=options)
    
    team_data = {}

    for team_id in team_ids:
        try:
            if category == "statcast":
                url = f"https://baseballsavant.mlb.com/team/{team_id}?view={category}&nav={analysis_type}&season={year}"
                datatable_id = 'div_statcast'
            elif category == "gamelogs":
                url = f"https://baseballsavant.mlb.com/team/{team_id}?view={category}&nav={analysis_type}&season={year}"
                datatable_id = 'div_gamelogs'

            driver.get(url)
            
            datatable_xpath = f"//div[@id='{datatable_id}']"

            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, datatable_xpath))
            )

            table_element = driver.find_element(By.XPATH, datatable_xpath)
            text_content = table_element.text

            rows = text_content.split("\n")
            table_data = [row.split("\t") for row in rows]

            team_data[team_id] = pd.DataFrame(table_data)
            
            # Print the status of the data retrieval
            if category == "statcast":
                print(f"{analysis_type} data successfully retrieved for team {team_id}")
            elif category == "gamelogs":
                print(f"{analysis_type} gamelogs successfully retrieved for team {team_id}")

        except Exception as e:
            print(f"Skipping team {team_id} due to error: {e}")
            continue  # Skip to the next iteration
        
    driver.quit()
    return team_data

# Import the tables for each team
team_ids = [108, 117, 133, 141, 144, 158, 138, 112, 109, 119, 137, 114,
            137, 114, 136, 146, 121, 120, 110, 135, 143, 134, 140, 139,
            113, 111, 115, 118, 116, 142, 145, 147]

# Import the advanced stats for each team
data_pitching = import_advanced_stats("pitching", team_ids, "statcast")
data_hitting = import_advanced_stats("hitting", team_ids, "statcast")

# Import the gamelogs for each team
gamelog_data_pitching = import_advanced_stats("pitching", team_ids, "gamelogs")
gamelog_data_hitting = import_advanced_stats("hitting", team_ids, "gamelogs")

pitching data successfully retrieved for team 108
pitching data successfully retrieved for team 117
pitching data successfully retrieved for team 133
pitching data successfully retrieved for team 141
pitching data successfully retrieved for team 144
pitching data successfully retrieved for team 158
pitching data successfully retrieved for team 138
pitching data successfully retrieved for team 112
pitching data successfully retrieved for team 109
pitching data successfully retrieved for team 119
pitching data successfully retrieved for team 137
pitching data successfully retrieved for team 114
pitching data successfully retrieved for team 137
pitching data successfully retrieved for team 114
pitching data successfully retrieved for team 136
pitching data successfully retrieved for team 146
pitching data successfully retrieved for team 121
pitching data successfully retrieved for team 120
pitching data successfully retrieved for team 110
pitching data successfully retrieved for team 135


In [ ]:
def team_advanced_stats(dataframe_name):
    """Get the advanced stats for the team.
    The function scrapes the advanced stats from the MLB website using Selenium and returns three dataframes:
    statcast, plate_discipline and batted_ball_profile.

    Returns:
        _type_: _description_
    """   
        
    #! STATCAST TABLE
    #! Cleaning the data for the statcast table
    def combine_rows(df, row_number_1, row_number_2):
        """Combine two rows in a DataFrame into one row.
        The first row will contain the combined data, and the second row will be dropped.

        Args:
            df (_type_): _description_
            row_number_1 (_type_): _description_
            row_number_2 (_type_): _description_

        Returns:
            _type_: _description_
        """
        # Combine the two rows
        df.loc[row_number_1, 0] = df.loc[row_number_1, 0] + ' ' + df.loc[row_number_2, 0]

        # Drop the second row
        df = df.drop(row_number_2).reset_index(drop=True)
        
        return df


    # Join rows
    dataframe_name = combine_rows(dataframe_name, 19, 20)
    dataframe_name = combine_rows(dataframe_name, 23, 24)
    dataframe_name = combine_rows(dataframe_name, 24, 25)

    #! Create the headers for the first table (statcast)
    # Find the first occurrence of 'Player' and 'XWOBACON'
    # Find indices
    start_idx = dataframe_name[dataframe_name[0] == 'Player'].index[0]
    end_idx = dataframe_name[dataframe_name[0] == 'XWOBACON'].index[0]

    # Slice and transpose
    headers_statcast = dataframe_name.iloc[start_idx:end_idx + 1].T

    # Reset column names
    headers_statcast.columns = headers_statcast.iloc[0]
    headers_statcast         = headers_statcast[1:].reset_index(drop=True)

    # Remove those rows from original dataframe
    dataframe_name = dataframe_name.drop(dataframe_name.index[start_idx:end_idx + 1]).reset_index(drop=True)

    # Remove the first 3 rows
    dataframe_name = dataframe_name.iloc[3:].reset_index(drop=True)  # Using iloc

    # Use regex to split the column while preserving negative numbers
    dataframe_name[['Name', 'Numbers']] = dataframe_name[0].str.extract(r'^(.*?)([-\d\s.,]*)$')
    
    # Remove initial dot from values that start with "."
    dataframe_name['Numbers'] = dataframe_name['Numbers'].apply(lambda x: x[1:] if x.startswith('. ') else x)

    # Split the 'Numbers' column into separate columns (25 columns)
    dataframe_name = dataframe_name.join(dataframe_name['Numbers'].str.split(expand=True).rename(lambda x: f'col_{x+1}', axis=1))

    # Drop the original 'Numbers' column
    dataframe_name = dataframe_name.drop(columns=['Numbers'])

    # Find the first empty row
    first_empty_idx = dataframe_name[dataframe_name[0] == ''].index.min()

    # Extract rows from the start until the first empty row
    statcast = dataframe_name.iloc[:first_empty_idx]

    # Drop the first column
    statcast = statcast.drop(columns=[0])

    # Swap last name and first name
    statcast['Name'] = statcast['Name'].str.split(', ').str[::-1].str.join(' ')

    # Add the headers to the dataframe
    statcast.columns = headers_statcast.columns

    #! PLATE DISCIPLINE TABLE
    #! Cleaning the data for the plate discipline table
    # Find the first empty row
    first_empty_idx = dataframe_name[dataframe_name[0] == ''].index.min()

    # Remove rows from the first row until the first empty row
    dataframe_name = dataframe_name.iloc[first_empty_idx + 1:].reset_index(drop=True)

    # Remove the first row
    dataframe_name = dataframe_name.iloc[1:].reset_index(drop= True)  # Using iloc

    # Join rows
    dataframe_name = combine_rows(dataframe_name, 4, 5)
    dataframe_name = combine_rows(dataframe_name, 5, 6)
    dataframe_name = combine_rows(dataframe_name, 7, 8)
    dataframe_name = combine_rows(dataframe_name, 9, 10)
    dataframe_name = combine_rows(dataframe_name, 13, 14)

    #! Create the headers for the second table (plate discipline)
    # Find the first occurrence of 'Player' and 'Meatball Swing %'
    # Find indices
    start_idx = dataframe_name[dataframe_name[0] == 'Player'].index[0]
    end_idx = dataframe_name[dataframe_name[0] == 'Meatball Swing %'].index[0]

    # Slice and transpose
    headers_plate_discipline = dataframe_name.iloc[start_idx:end_idx + 1].T

    # Reset column names
    headers_plate_discipline.columns = headers_plate_discipline.iloc[0]
    headers_plate_discipline         = headers_plate_discipline[1:].reset_index(drop=True)

    # Remove those rows from original dataframe
    dataframe_name = dataframe_name.drop(dataframe_name.index[start_idx:end_idx + 1]).reset_index(drop=True)

    # Find the first empty row
    first_empty_idx = dataframe_name[dataframe_name[0] == ''].index.min()

    # Extract rows from the start until the first empty row
    plate_discipline = dataframe_name.iloc[:first_empty_idx]

    # Drop the first column
    plate_discipline = plate_discipline.drop(columns=[0])

    # Swap last name and first name
    plate_discipline['Name'] = plate_discipline['Name'].str.split(', ').str[::-1].str.join(' ')

    # Drop columns where all values are NaN (empty)
    plate_discipline = plate_discipline.dropna(axis= 1, how= 'all')

    # Add the headers to the dataframe
    plate_discipline.columns = headers_plate_discipline.columns

    #! BATTED BALL PROFILE TABLE
    #! Cleaning the data for the batted ball profile table
    # Find the first empty row
    first_empty_idx = dataframe_name[dataframe_name[0] == ''].index.min()

    # Remove rows from the first row until the first empty row
    dataframe_name = dataframe_name.iloc[first_empty_idx + 1:].reset_index(drop=True)

    # Remove the first row
    dataframe_name = dataframe_name.iloc[1:].reset_index(drop= True)  # Using iloc

    # Join rows
    dataframe_name = combine_rows(dataframe_name, 15, 16)

    #! Create the headers for the third table (batted ball profile)
    # Find the first occurrence of 'Player' and 'Barrel %'
    # Find indices
    start_idx = dataframe_name[dataframe_name[0] == 'Player'].index[0]
    end_idx = dataframe_name[dataframe_name[0] == 'Barrel %'].index[0]

    # Slice and transpose
    headers_batted_ball_profile = dataframe_name.iloc[start_idx:end_idx + 1].T

    # Reset column names
    headers_batted_ball_profile.columns = headers_batted_ball_profile.iloc[0]
    headers_batted_ball_profile         = headers_batted_ball_profile[1:].reset_index(drop=True)

    # Remove those rows from original dataframe
    dataframe_name = dataframe_name.drop(dataframe_name.index[start_idx:end_idx + 1]).reset_index(drop=True)

    # Find the first empty row
    first_empty_idx = dataframe_name[dataframe_name[0] == ''].index.min()

    if pd.isna(first_empty_idx):
        batted_ball_profile = dataframe_name.copy()  # If no empty row, use the entire DataFrame
    else:
        # Extract rows from the start until the first empty row
        batted_ball_profile = dataframe_name.iloc[:first_empty_idx]

    # Drop the first column
    batted_ball_profile = batted_ball_profile.drop(columns=[0])

    # Swap last name and first name
    batted_ball_profile['Name'] = batted_ball_profile['Name'].str.split(', ').str[::-1].str.join(' ')

    # Drop columns where all values are NaN (empty)
    batted_ball_profile = batted_ball_profile.dropna(axis= 1, how= 'all')

    # Add the headers to the dataframe
    batted_ball_profile.columns = headers_batted_ball_profile.columns
    
    return statcast, plate_discipline, batted_ball_profile


In [ ]:
def process_gamelogs(dataframe_name):        
    #! Create the headers for the gamelogs
    # Find the first occurrence of 'Game Date' and 'Hard Hit %'
    # Find indices
    start_idx = dataframe_name[dataframe_name[0] == 'Game Date'].index[0]
    end_idx = dataframe_name[dataframe_name[0] == 'Hard Hit %'].index[0]

    # Slice and transpose
    headers_gamelogs = dataframe_name.iloc[start_idx:end_idx + 1].T

    # Reset column names
    headers_gamelogs.columns = headers_gamelogs.iloc[0]
    headers_gamelogs         = headers_gamelogs[1:].reset_index(drop=True)
    
    # Remove those rows from original dataframe
    dataframe_name = dataframe_name.drop(dataframe_name.index[start_idx:end_idx + 1]).reset_index(drop=True)
    
    # Use regex to split the column while preserving negative numbers
    dataframe_name[['Name', 'Numbers']] = dataframe_name[0].str.extract(r'^(.*?)([-\d\s.,]*)$')
        
    # Split the 'Numbers' column into separate columns (25 columns)
    dataframe_name = dataframe_name.join(dataframe_name['Numbers'].str.split(expand=True).rename(lambda x: f'col_{x+1}', axis=1))
    
    # Drop the first column
    dataframe_name = dataframe_name.drop(columns=[0])
    
    # Drop the original 'Numbers' column
    dataframe_name = dataframe_name.drop(columns=['Numbers'])
    
    # Remove the first 2 rows
    dataframe_name = dataframe_name.iloc[2:].reset_index(drop= True)  # Using iloc
    
    # # Split the 'Name' column into 'Game Date' and 'Opponent'
    # dataframe_name[['Game Date', 'Opponent']] = dataframe_name['Name'].str.split(" ", n=2, expand=True)
    
    # Create a copy of the dataframe to store the gamelogs
    gamelogs = dataframe_name.copy()
    
    # Split the first column into two
    gamelogs[['Game Date', 'Opponent']] = gamelogs['Name'].str.split(" ", n=1, expand=True)
    
    # Drop the first column
    gamelogs = gamelogs.drop(columns= ['Name'])
    
    # Get the last two column names
    last_two_cols  = gamelogs.columns[-2:].tolist()
    remaining_cols = gamelogs.columns[:-2].tolist()

    # Reorder the DataFrame
    gamelogs = gamelogs[last_two_cols + remaining_cols]
    
    # Add the headers to the dataframe
    gamelogs.columns = headers_gamelogs.columns

    return gamelogs

### Apply a function to the dictionary


In [ ]:

#! STATCAST TABLES
# Initialize dictionaries to store the results for each team
statcast_data_pitching            = {}
plate_discipline_data_pitching    = {}
batted_ball_profile_data_pitching = {}

statcast_data_hitting             = {}
plate_discipline_data_hitting     = {}
batted_ball_profile_data_hitting  = {}

#? Pitching
# Iterate over the team_data dictionary
for team_id, df in data_pitching.items():
    # Call the team_advanced_stats function and unpack the returned DataFrames
    statcast, plate_discipline, batted_ball_profile = team_advanced_stats(df)
    
    # Store the results in the respective dictionaries
    statcast_data_pitching[team_id]            = statcast
    plate_discipline_data_pitching[team_id]    = plate_discipline
    batted_ball_profile_data_pitching[team_id] = batted_ball_profile

#? Hitting
# Iterate over the team_data dictionary
for team_id, df in data_hitting.items():
    # Call the team_advanced_stats function and unpack the returned DataFrames
    statcast, plate_discipline, batted_ball_profile = team_advanced_stats(df)
    
    # Store the results in the respective dictionaries
    statcast_data_hitting[team_id]            = statcast
    plate_discipline_data_hitting[team_id]    = plate_discipline
    batted_ball_profile_data_hitting[team_id] = batted_ball_profile
    

#! GAMELOGS
# Initialize dictionaries to store the results for each team
gamelog_pitching = {}
gamelog_hitting  = {}

#? Pitching
# Iterate over the team_data dictionary
for team_id, df in gamelog_data_pitching.items():
    # Call the team_advanced_stats function and unpack the returned DataFrames
    gamelogs = process_gamelogs(df)
    
    # Store the results in the respective dictionaries
    gamelog_pitching[team_id] = gamelogs

#? Hitting
# Iterate over the team_data dictionary
for team_id, df in gamelog_data_hitting.items():
    # Call the team_advanced_stats function and unpack the returned DataFrames
    gamelogs = process_gamelogs(df)
    
    # Store the results in the respective dictionaries
    gamelog_hitting[team_id] = gamelogs

In [ ]:
def create_parquet_files(data_dict, suffix):
    """Create Parquet files from a dictionary of DataFrames."""

    # Create new DataFrames with a suffix
    df_store = {}

    for idx, (key, df) in enumerate(data_dict.items(), start= 1):
        new_name = f"{key}{suffix}"  # Assigning a numbered suffix
        df_store[new_name] = df.copy()  # Storing the DataFrame in a dictionary
        
    for name, df in df_store.items():
        # Save hitting statcast DataFrames
        df.to_parquet(f"D:\\mlb_analyzer\\output\\teams\\{suffix}\\{name}.parquet")


# Call the function and export the DataFrames to Parquet files
create_parquet_files(statcast_data_hitting, "_hitting_statcast")
create_parquet_files(plate_discipline_data_hitting, "_hitting_plate_discipline")
create_parquet_files(batted_ball_profile_data_hitting, "_hitting_batted_ball_profile")

create_parquet_files(statcast_data_pitching, "_pitching_statcast")
create_parquet_files(plate_discipline_data_pitching, "_pitching_plate_discipline")
create_parquet_files(batted_ball_profile_data_pitching, "_pitching_batted_ball_profile")

create_parquet_files(gamelog_hitting, "_gamelogs_hitting")
create_parquet_files(gamelog_pitching, "_gamelogs_pitching")